In [7]:
import os
import openpyxl
import boto3
import pandas as pd
import io

In [8]:
bucket_name = 'trashback-data'
test_folder_key = 'sorted_images_test/'
train_folder_key = 'sorted_images/'
excel_file_key = 'waste_pics.xlsx'


s3 = boto3.client('s3')

excel_obj = s3.get_object(Bucket=bucket_name, Key=excel_file_key)
excel_data = pd.read_excel(io.BytesIO(excel_obj['Body'].read()))

excel_data.dropna(subset=['WASTE_TYPE'], inplace=True)

In [22]:
from data_processing import list_keys

keys = list_keys(bucket_name,train_folder_key)
keys_test = list_keys(bucket_name,test_folder_key)  

In [11]:
categories = excel_data['WASTE_TYPE'].unique()


In [23]:
dict_of_images = {}
dict_filenames = {}
dict_of_categories = {}

for category in categories:
    dict_of_categories[category] = []

for key in keys:
    filename = key.split('/')[-1]
    category,sub_category = key.split('/')[-3],key.split('/')[-2]
    dict_of_images[key] = category + '/' + sub_category
    dict_filenames[filename] = category #+ '/' + sub_category
    if category in categories:

        dict_of_categories[category].append(filename)

for key in keys_test:
    filename = key.split('/')[-1]
    category,sub_category = key.split('/')[-3],key.split('/')[-2]
    dict_of_images[key] = category + '/' + sub_category
    dict_filenames[filename] = category #+ '/' + sub_category
    if category in categories:

        dict_of_categories[category].append(filename)

In [18]:
# i want to count the number of filenames in each category with the dict_of_categories

for category in categories:
    print(category, len([key for key in dict_filenames.keys() if dict_filenames[key] == category]))

Plastique 10800
Mégots 1780
Papier ou Carton 2435
Textile 795
Métaux 6135
Verre 2058
Matières Organiques 111
nan 0


In [46]:
for category in categories:
    print(category, len(dict_of_categories[category]))

Plastique 999
Mégots 999
Papier ou Carton 999
Textile 948
Métaux 999
Verre 999
Matières Organiques 275
nan 0


In [29]:
for category in categories:
    dict_of_categories[category].pop(0)

IndexError: pop from empty list

In [39]:
# we select all the biowaste

biowaste = [excel_data['PIC_NAME'][i] for i in range(len(excel_data['WASTE_TYPE'])) if excel_data['WASTE_TYPE'][i] == 'Matières Organiques']
    

In [49]:
dir_path = "/Users/macbook/Desktop/Trashback1000"
original_dir = "/Users/macbook/Desktop/Trashback"

#copy the selected files into the folder in dir_path

import shutil
'''
for category in categories:
    os.mkdir(dir_path + '/' + category)'''

for category in categories:
    for filename in dict_of_categories[category]:
        if filename != '':
            shutil.copy(original_dir + '/' + filename, dir_path + '/' + category + '/' + filename)

In [ ]:
#empty the folders of the dir_path

for category in categories:
    for filename in dict_of_categories[category]:
        if filename != '':
            os.remove(dir_path + '/'+ filename)

In [53]:
!aws s3 cp /Users/macbook/Desktop/Trashback1000 s3://trashback-data/trashback1000 --recursive

upload: ../../../../Trashback1000/.DS_Store to s3://trashback-data/trashback1000/.DS_Store
upload: ../../../../Trashback1000/Matières Organiques/1690442774346-user.jpg to s3://trashback-data/trashback1000/Matières Organiques/1690442774346-user.jpg
upload: ../../../../Trashback1000/Matières Organiques/1690460137723-image1690460053578.jpg to s3://trashback-data/trashback1000/Matières Organiques/1690460137723-image1690460053578.jpg
upload: ../../../../Trashback1000/Matières Organiques/1689954925080-image1689954931709.jpg to s3://trashback-data/trashback1000/Matières Organiques/1689954925080-image1689954931709.jpg
upload: ../../../../Trashback1000/Matières Organiques/1690523230919-image1690523230826.jpg to s3://trashback-data/trashback1000/Matières Organiques/1690523230919-image1690523230826.jpg
upload: ../../../../Trashback1000/Matières Organiques/1690523275960-image1690523276792.jpg to s3://trashback-data/trashback1000/Matières Organiques/1690523275960-image1690523276792.jpg
upload: ../.